# Chapter 03 실습 1: Optimizer 비교

## 목표
동일한 Fashion MNIST 분류 모델에 SGD, Adam, RMSprop을 적용하고 수렴 속도를 비교한다.

## 실습 내용
- 동일한 가중치 초기화로 3개의 동일한 모델 생성
- 각 옵티마이저로 동일한 에포크 동안 학습
- 훈련 손실, 검증 손실, 검증 정확도를 한 그래프에 비교
- 결과를 분석하고 각 옵티마이저의 특성을 이해

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sys
import time

# 상위 디렉토리 경로 추가 (공통 유틸 사용 시)
sys.path.append('..')

print("TensorFlow 버전:", tf.__version__)
print("NumPy 버전:", np.__version__)

# 재현성을 위한 시드 고정
SEED = 42
tf.random.set_seed(SEED)
np.random.seed(SEED)

## 실험 설계

공정한 비교를 위해 다음 조건을 통일한다:

| 항목 | 설정값 |
|------|--------|
| 데이터셋 | Fashion MNIST |
| 모델 구조 | Dense(256, relu) -> Dropout(0.3) -> Dense(128, relu) -> Dense(10, softmax) |
| 가중치 초기화 | 동일한 시드로 동일하게 초기화 |
| 에포크 수 | 20 |
| 배치 크기 | 128 |
| 손실 함수 | SparseCategoricalCrossentropy |
| **변수** | **옵티마이저** (SGD, Adam, RMSprop) |

각 옵티마이저의 학습률:
- SGD: lr=0.01 (Momentum=0.9)
- Adam: lr=0.001 (기본값)
- RMSprop: lr=0.001 (기본값)

In [ ]:
# ---------------------------------------------------
# 데이터 준비: Fashion MNIST
# ---------------------------------------------------

# Fashion MNIST 로드
(X_train_full, y_train_full), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

# 정규화: [0, 255] -> [0, 1]
X_train_full = X_train_full.reshape(-1, 784).astype('float32') / 255.0
X_test = X_test.reshape(-1, 784).astype('float32') / 255.0

# 훈련/검증 분리
X_train, X_val = X_train_full[:50000], X_train_full[50000:]
y_train, y_val = y_train_full[:50000], y_train_full[50000:]

# 클래스 레이블
CLASS_NAMES = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

print("데이터셋 정보:")
print(f"  훈련 셋: {X_train.shape} | 레이블: {y_train.shape}")
print(f"  검증 셋: {X_val.shape}   | 레이블: {y_val.shape}")
print(f"  테스트 셋: {X_test.shape} | 레이블: {y_test.shape}")
print(f"  클래스 수: {len(CLASS_NAMES)}")
print(f"  클래스: {CLASS_NAMES}")

In [ ]:
# ---------------------------------------------------
# 동일 구조 모델 생성 함수 (동일한 가중치 초기화)
# ---------------------------------------------------

def create_model(optimizer, seed=42):
    """동일한 구조와 초기화로 모델을 생성하고 컴파일한다.
    
    Args:
        optimizer: tf.keras.optimizers 인스턴스
        seed: 가중치 초기화 시드 (동일한 초기화 보장)
    
    Returns:
        컴파일된 tf.keras.Model
    """
    # 시드를 고정하여 동일한 초기 가중치 보장
    tf.random.set_seed(seed)
    
    # 동일한 initializer 사용
    initializer = tf.keras.initializers.GlorotUniform(seed=seed)
    
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(
            256, activation='relu',
            input_shape=(784,),
            kernel_initializer=initializer
        ),
        tf.keras.layers.Dropout(0.3, seed=seed),
        tf.keras.layers.Dense(
            128, activation='relu',
            kernel_initializer=initializer
        ),
        tf.keras.layers.Dense(
            10, activation='softmax',
            kernel_initializer=initializer
        )
    ])
    
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model


# 실험할 옵티마이저 설정
EPOCHS = 20
BATCH_SIZE = 128

optimizer_configs = {
    'SGD (lr=0.01, momentum=0.9)': tf.keras.optimizers.SGD(
        learning_rate=0.01, momentum=0.9
    ),
    'Adam (lr=0.001)': tf.keras.optimizers.Adam(
        learning_rate=0.001
    ),
    'RMSprop (lr=0.001)': tf.keras.optimizers.RMSprop(
        learning_rate=0.001
    ),
}

print("실험 설정:")
print(f"  에포크: {EPOCHS}")
print(f"  배치 크기: {BATCH_SIZE}")
print(f"  비교할 옵티마이저: {list(optimizer_configs.keys())}")

In [ ]:
# ---------------------------------------------------
# 각 옵티마이저로 학습 실행
# ---------------------------------------------------

all_histories = {}
training_times = {}

for opt_name, optimizer in optimizer_configs.items():
    print(f"\n{'='*50}")
    print(f"학습 중: {opt_name}")
    print(f"{'='*50}")
    
    # 동일한 초기 가중치로 새 모델 생성
    model = create_model(optimizer, seed=SEED)
    
    start_time = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(X_val, y_val),
        verbose=0  # 진행 상황 숨김
    )
    elapsed = time.time() - start_time
    
    all_histories[opt_name] = history.history
    training_times[opt_name] = elapsed
    
    # 결과 요약
    final_val_acc = history.history['val_accuracy'][-1]
    best_val_acc = max(history.history['val_accuracy'])
    best_epoch = history.history['val_accuracy'].index(best_val_acc) + 1
    
    print(f"  학습 시간: {elapsed:.1f}초")
    print(f"  최종 검증 정확도: {final_val_acc:.4f}")
    print(f"  최고 검증 정확도: {best_val_acc:.4f} (에포크 {best_epoch})")

print("\n모든 옵티마이저 학습 완료!")

In [ ]:
# ---------------------------------------------------
# 학습 곡선 비교 시각화
# ---------------------------------------------------

# 색상 설정
colors = {
    'SGD (lr=0.01, momentum=0.9)': 'blue',
    'Adam (lr=0.001)': 'red',
    'RMSprop (lr=0.001)': 'green',
}

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
epochs_range = range(1, EPOCHS + 1)

# 1. 훈련 손실
ax = axes[0, 0]
for name, history in all_histories.items():
    ax.plot(epochs_range, history['loss'], 
            color=colors[name], linewidth=2, label=name)
ax.set_xlabel('에포크')
ax.set_ylabel('손실')
ax.set_title('훈련 손실 비교')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 2. 검증 손실
ax = axes[0, 1]
for name, history in all_histories.items():
    ax.plot(epochs_range, history['val_loss'],
            color=colors[name], linewidth=2, label=name)
ax.set_xlabel('에포크')
ax.set_ylabel('손실')
ax.set_title('검증 손실 비교')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 3. 훈련 정확도
ax = axes[1, 0]
for name, history in all_histories.items():
    ax.plot(epochs_range, history['accuracy'],
            color=colors[name], linewidth=2, label=name)
ax.set_xlabel('에포크')
ax.set_ylabel('정확도')
ax.set_title('훈련 정확도 비교')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

# 4. 검증 정확도
ax = axes[1, 1]
for name, history in all_histories.items():
    ax.plot(epochs_range, history['val_accuracy'],
            color=colors[name], linewidth=2, label=name)
ax.set_xlabel('에포크')
ax.set_ylabel('정확도')
ax.set_title('검증 정확도 비교')
ax.legend(fontsize=8)
ax.grid(True, alpha=0.3)

plt.suptitle('Fashion MNIST - 옵티마이저별 학습 곡선 비교', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# 최종 결과 표
print("\n=== 최종 결과 비교 ===")
print(f"{'옵티마이저':<35} {'최고 Val Acc':^15} {'최종 Val Acc':^15} {'학습 시간':^12}")
print("-" * 80)
for name, history in all_histories.items():
    best_acc = max(history['val_accuracy'])
    final_acc = history['val_accuracy'][-1]
    t_time = training_times[name]
    print(f"{name:<35} {best_acc:^15.4f} {final_acc:^15.4f} {t_time:^12.1f}초")

## 결과 해석 가이드

### 일반적으로 기대되는 결과

1. **Adam**: 초기 수렴이 가장 빠름. 적응적 학습률로 초기 몇 에포크 내에 빠르게 수렴한다.

2. **RMSprop**: Adam과 비슷하게 빠른 수렴을 보이나, Adam보다 약간 안정성이 낮을 수 있다.

3. **SGD + Momentum**: 초반에 느리지만, 많은 에포크를 학습할수록 Adam과 격차가 줄어들거나 역전될 수 있다. 잘 튜닝된 SGD가 최고 성능을 보이는 경우도 많다.

### 그래프 해석 포인트

- **손실 감소 속도**: 초기 에포크에서 어떤 옵티마이저가 더 빨리 수렴하는가?
- **수렴 안정성**: 손실 곡선이 얼마나 smooth한가? 진동이 있는가?
- **과적합**: 훈련 손실과 검증 손실의 차이가 크면 과적합 신호
- **최종 성능**: 20 에포크 후 어떤 옵티마이저가 가장 높은 검증 정확도를 달성하는가?

## 도전 과제

1. **학습률 튜닝**: SGD의 학습률을 0.01, 0.05, 0.001로 변경하여 어떤 값이 최적인지 찾아보세요.

2. **에포크 증가**: EPOCHS를 50으로 늘려서 더 오래 학습했을 때 결과가 어떻게 변하는지 확인하세요.

3. **AdamW 추가**: `tf.keras.optimizers.AdamW`를 실험에 추가하고 Adam과 비교해보세요.

4. **배치 크기 변경**: BATCH_SIZE를 32, 64, 256으로 변경하고 수렴 속도 차이를 관찰하세요.

5. **Learning Rate Scheduler 적용**: ExponentialDecay를 SGD에 적용하여 Adam과의 격차를 줄여보세요.